# Comparing Expert, MAB, FB and FR algorithms performances

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import sys
sys.path.append('../')

In [3]:
import source.players as pl
import source.parsers as parsers
import source.game as game
import source.player as player
import source.environment as environment
import source.errors as errors
import source.runner as runner
import source.util as util
import source.players.attackers as attackers
import source.players.base_defenders as base_defenders
import source.players.defenders as defenders
import source.players.holmes as holmes
import source.players.sad as sad
import source.players.b2bw2w as b2bw2w
import source.players.belief_max as bm
import source.players.dmd as dmd

import concurrent.futures
import pkgutil
import logging
import sys
from source.util import *
import numpy as np
import pandas as pd
from importlib import *
import time
import os
import re
from copy import deepcopy, copy
import datetime
from contextlib import redirect_stdout
from random import shuffle

In [4]:
# run once

lvl = logging.DEBUG
logger = logging.getLogger("source")
logger.setLevel(lvl)
#ch = logging.StreamHandler(sys.stdout)
ch = logging.FileHandler("status", mode='w')
ch.setLevel(lvl)
#formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
formatter = logging.Formatter('%(message)s')
ch.setFormatter(formatter)
logger.addHandler(ch)

In [5]:
lvl = logging.WARNING
logger.setLevel(lvl)
ch.setLevel(lvl)

In [6]:
# if you change something in the files, don't forget to reload
reload(player)
reload(base_defenders)
reload(attackers)
reload(defenders)
reload(errors)
reload(environment)
reload(runner)
reload(parsers)
reload(game)
reload(holmes)
reload(sad)
reload(b2bw2w)
reload(util)
reload(bm)
reload(dmd)

<module 'source.players.dmd' from '..\\source\\players\\dmd.py'>

### Creating configurations to save into a csv file

In [7]:
time_horizon = 100
T = 6
#targets = util.gen_targets(T, low=1, high=10)
targets = util.gen_tar_with_len(T)
values = tuple((v, v) for v in targets)
print(values)
#obs = gen_observabilities_correlated_with_values(values)
#obs = {0: 0.75, 1: 0.75, 2: 0.75, 3: 0.75, 4: 0.75}
#obs_vect = [obs.get(i) for i in range(len(targets))]
#generating pure strat and some rand mixed strat stochastic attackers
distributions = []
for i in range(10):
    distributions.append(tuple(util.gen_distr(T)))
print(distributions)
chosen_distr = tuple(util.gen_distr(T))
print("chosen = ", chosen_distr)

((0.783, 0.783), (0.871, 0.871), (0.947, 0.947), (0.942, 0.942), (0.952, 0.952), (0.997, 0.997))
[(0.07807807807807808, 0.03803803803803804, 0.03303303303303304, 0.07707707707707709, 0.05605605605605606, 0.7177177177177178), (0.029970029970029965, 0.5064935064935064, 0.07492507492507491, 0.004995004995004994, 0.2807192807192807, 0.10289710289710288), (0.085, 0.27, 0.166, 0.035, 0.333, 0.111), (0.16983016983016982, 0.05694305694305694, 0.10889110889110888, 0.1978021978021978, 0.4325674325674325, 0.03396603396603397), (0.20779220779220775, 0.00899100899100899, 0.08591408591408589, 0.2627372627372627, 0.21478521478521476, 0.21978021978021975), (0.06093906093906094, 0.49350649350649356, 0.26173826173826176, 0.03796203796203797, 0.1398601398601399, 0.005994005994005995), (0.6616616616616616, 0.03603603603603603, 0.17717717717717715, 0.01001001001001001, 0.1051051051051051, 0.01001001001001001), (0.17217217217217218, 0.30630630630630634, 0.06206206206206207, 0.0840840840840841, 0.29029029029

In [8]:
g = game.Game(values, time_horizon)
g.attackers = [1]
g.defenders = [0]
d = [dmd.DirichletMultinomialDefender(g, 0)]
prof = []
for distr in distributions:
    prof.append(attackers.StochasticAttacker(g, 1, 1, *distr))
for i in range(10):
    prof.append(attackers.SUQR(g,1))
chosen_a_prof = (attackers.StochasticAttacker(g, 1, 1, *chosen_distr))
a = [deepcopy(chosen_a_prof)]
prof.append(chosen_a_prof)
shuffle(prof)
print("Real attacker is: ", chosen_a_prof)
g.set_players(d, a, prof)

Real attacker is:  sto1-0.21-0.054-0.319-0.016-0.143-0.258


In [13]:
def print_header(targets, profiles):
    return "Name,T," + ",".join(str(i) for i in range(len(targets))) + ",Defender,Attacker," + ",".join(["Profile" for x in range(len(profiles))]) + "\n" 

In [14]:
def print_row(targets, time_horizon, d, a, p):
    return ",".join([str(i) for i in ([d+"_vs_"+a] + [time_horizon] + targets + [d] + [a] + [x for x in p]) ])+"\n"

In [15]:
(print_header(targets, prof))

'Name,T,0,1,2,3,4,5,Defender,Attacker,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile,Profile\n'

In [16]:
defender =["FB1"]

In [17]:
profile_strings = []
for p in prof:
    profile_strings.append(util.print_adv(p))

In [7]:
experiments_folder = "../NewExperiments"
exp_subfolder = "/Bayesian vs FB"
batch_name = "fb"
batch_file = batch_name + ".csv"
batch_path = experiments_folder + exp_subfolder + "/" + batch_file

### Generating the csv and checking everything is fine

In [20]:
with open(batch_path, "w+") as f:
    f.write(print_header(targets, prof))
    f.write(print_row(targets, time_horizon,defender[0],util.print_adv(a[0]),profile_strings))

In [21]:
pd.read_csv(batch_path)

,Name,T,0,1,2,3,4,5,Defender,Attacker,...,Profile.16,Profile.17,Profile.18,Profile.19,Profile.20,Profile.21,Profile.22,Profile.23,Profile.24,Profile.25
0,DMD1_vs_sto1-0.044-0.046-0.063-0.204-0.159-0.484,100,0.902,0.979,0.848,0.976,0.956,0.912,DMD1,sto1-0.044-0.046-0.063-0.204-0.159-0.484,...,sto1-0.221-0.062-0.137-0.3-0.026-0.254,sto1-0.037-0.016-0.234-0.052-0.06-0.601,sto1-1-0-0-0-0-0,sto1-0.073-0.032-0.459-0.089-0.281-0.066,sto1-0.050050050050050046-0.02302302302302302-...,sto1-0.242-0.182-0.053-0.133-0.192-0.198,sto1-0-0-0-0-0-1,sto1-0.018-0.358-0.066-0.437-0.089-0.032,sto1-0.078-0.43-0.038-0.006-0.218-0.23,sto1-0.6796796796796797-0.10910910910910909-0....


### Parsing the csv to generate a batch containing the configurations and running them

In [8]:
#os.mkdir("../NewExperiments/Exp2/belief_max_obs_vs_no_obs")

In [9]:
b = runner.Batch(batch_path, experiments_folder + exp_subfolder)

In [10]:
b.parse_batch()

In [11]:
batches = []
batches.append(b)

In [12]:
with concurrent.futures.ProcessPoolExecutor(None) as executor:
    futures = {}
    logger.info(str(datetime.datetime.now()))
    for b in batches:
        b.run(futures, executor, n=2)

### Collecting results of the runs

In [13]:
with open('progress', 'w') as f:
    with redirect_stdout(f):
        for b in batches:
            b.collect(futures)
            confs = b.configurations
            print(b.results_folder_path)
            dlist = [{"name": c.game.players[0].__class__.name,
                      "avgs": c.stats["exp_regret"], 
                      "lb": c.stats["lb_exp_regret"],
                      "ub": c.stats["ub_exp_regret"]}
                     for c in confs]
            util.plot_dicts(dlist, save=True, show=False, semilog=True,
                            path=b.results_folder_path,
                            title=confs[0].game.players[1].__class__.name)

### Checking everything was fine by running the two configurations "step-by-step"

In [14]:
gm = deepcopy(batches[0].configurations[0].game)

In [15]:
gm

<Game values:[[0.902, 0.902], [0.9790000000000001, 0.9790000000000001], [0.848, 0.848], [0.976, 0.976], [0.956, 0.956], [0.912, 0.912]] players{1: <StochasticAttacker id:1 resources:1>, 0: <DirichletMultinomialDefender id:0 resources:1>} time_horizon:100>

In [16]:
e = runner.Experiment(gm)

In [17]:
print(gm.players[0].profiles[0])

sto1-0.06693306693306694-0.12687312687312688-0.2667332667332668-0.04695304695304696-0.2577422577422578-0.23476523476523478


In [18]:
for i in range(900):
    e.run_interaction()
    print(e.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e.game.history[-1])
    #print(list(e.agent.belief.pr.items())[2])

[0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666, 0.16666666666666666]
sto1-0.06693306693306694-0.12687312687312688-0.2667332667332668-0.04695304695304696-0.2577422577422578-0.23476523476523478 0.20052513258289173
sto1-0.4645354645354645-0.042957042957042946-0.049950049950049945-0.043956043956043946-0.10889110889110888-0.28971028971028967 0.4568821491143339
sto1-0.1781781781781782-0.3633633633633634-0.0930930930930931-0.04504504504504505-0.08808808808808809-0.23223223223223227 0.22507488381632706
sto1-0.018-0.358-0.066-0.437-0.089-0.032 0.6168516507622909
sto1-0.04304304304304304-0.050050050050050046-0.27727727727727725-0.06806806806806806-0.3643643643643643-0.1971971971971972 0.3321443212771846
sto1-0.6796796796796797-0.10910910910910909-0.027027027027027025-0.050050050050050046-0.09709709709709709-0.03703703703703703 0.6807523473591781
sto1-0.347-0.029-0.277-0.04-0.03-0.277 0.5235404000733983
sto1-0.281-0.096-0.134-0.295-0.108-0

sto1-0.2152152152152152-0.3973973973973974-0.02102102102102102-0.018018018018018014-0.15915915915915915-0.18918918918918917 1.1169790231876249
Best responding to: sto1-0.044-0.046-0.063-0.204-0.159-0.484 , with ent: 0.08656036447051521
{0: (0, 1, 0, 0, 0, 0), 1: [0.044, 0.046, 0.063, 0.204, 0.159, 0.484]}
{1: [2], 0: [1]}
[0.08, 0.04, 0.12, 0.32, 0.08, 0.36]
sto1-0.06693306693306694-0.12687312687312688-0.2667332667332668-0.04695304695304696-0.2577422577422578-0.23476523476523478 0.5466906027048744
sto1-0.4645354645354645-0.042957042957042946-0.049950049950049945-0.043956043956043946-0.10889110889110888-0.28971028971028967 0.6503787157240999
sto1-0.1781781781781782-0.3633633633633634-0.0930930930930931-0.04504504504504505-0.08808808808808809-0.23223223223223227 0.6556629862317216
sto1-0.018-0.358-0.066-0.437-0.089-0.032 0.8664945326743333
sto1-0.04304304304304304-0.050050050050050046-0.27727727727727725-0.06806806806806806-0.3643643643643643-0.1971971971971972 0.5308109708606428
sto1-0.

sto1-0.073-0.032-0.459-0.089-0.281-0.066 0.8533663243202336
sto1-0.141-0.006-0.137-0.096-0.246-0.374 0.26477864165991244
sto1-0.037-0.016-0.234-0.052-0.06-0.601 0.43152673625332716
sto1-0.078-0.43-0.038-0.006-0.218-0.23 1.3661373091545472
sto1-0.050050050050050046-0.02302302302302302-0.3973973973973974-0.22922922922922923-0.11711711711711711-0.18318318318318316 0.2910400400187087
sto1-0.06-0.304-0.044-0.454-0.136-0.002 1.9706621977828949
sto1-0.221-0.062-0.137-0.3-0.026-0.254 0.22586103491760495
sto1-0.038-0.683-0.074-0.045-0.131-0.029 1.5901466040148562
sto1-0.5785785785785785-0.04804804804804804-0.02402402402402402-0.016016016016016016-0.038038038038038034-0.29529529529529525 1.1445488164563509
sto1-0.2152152152152152-0.3973973973973974-0.02102102102102102-0.018018018018018014-0.15915915915915915-0.18918918918918917 1.1296357251680815
Best responding to: sto1-0.044-0.046-0.063-0.204-0.159-0.484 , with ent: 0.049561760586983764
{0: (0, 1, 0, 0, 0, 0), 1: [0.044, 0.046, 0.063, 0.204, 0

sto1-0.1781781781781782-0.3633633633633634-0.0930930930930931-0.04504504504504505-0.08808808808808809-0.23223223223223227 0.5580829474229496
sto1-0.018-0.358-0.066-0.437-0.089-0.032 0.9564182626429999
sto1-0.04304304304304304-0.050050050050050046-0.27727727727727725-0.06806806806806806-0.3643643643643643-0.1971971971971972 0.5671286515214431
sto1-0.6796796796796797-0.10910910910910909-0.027027027027027025-0.050050050050050046-0.09709709709709709-0.03703703703703703 1.3208109646023984
sto1-0.347-0.029-0.277-0.04-0.03-0.277 0.6483077237133035
sto1-0.281-0.096-0.134-0.295-0.108-0.086 0.44300355751060116
sto1-0.242-0.182-0.053-0.133-0.192-0.198 0.3054284570023146
sto1-0.044-0.046-0.063-0.204-0.159-0.484 0.05841462187007379
sto1-0.073-0.032-0.459-0.089-0.281-0.066 0.916404821332191
sto1-0.141-0.006-0.137-0.096-0.246-0.374 0.3208907812269874
sto1-0.037-0.016-0.234-0.052-0.06-0.601 0.43838473819184376
sto1-0.078-0.43-0.038-0.006-0.218-0.23 1.1722747790032155
sto1-0.050050050050050046-0.023023

sto1-0.06-0.304-0.044-0.454-0.136-0.002 2.119936051046711
sto1-0.221-0.062-0.137-0.3-0.026-0.254 0.17907899298028462
sto1-0.038-0.683-0.074-0.045-0.131-0.029 1.4979888761213327
sto1-0.5785785785785785-0.04804804804804804-0.02402402402402402-0.016016016016016016-0.038038038038038034-0.29529529529529525 0.9559124555870055
sto1-0.2152152152152152-0.3973973973973974-0.02102102102102102-0.018018018018018014-0.15915915915915915-0.18918918918918917 0.9201892134952692
Best responding to: sto1-0.044-0.046-0.063-0.204-0.159-0.484 , with ent: 0.06415686442046442
{0: (0, 1, 0, 0, 0, 0), 1: [0.044, 0.046, 0.063, 0.204, 0.159, 0.484]}
{1: [5], 0: [1]}
[0.06666666666666667, 0.08333333333333333, 0.06666666666666667, 0.26666666666666666, 0.06666666666666667, 0.45]
sto1-0.06693306693306694-0.12687312687312688-0.2667332667332668-0.04695304695304696-0.2577422577422578-0.23476523476523478 0.5380773411125375
sto1-0.4645354645354645-0.042957042957042946-0.049950049950049945-0.043956043956043946-0.10889110889

...

sto1-0.038-0.683-0.074-0.045-0.131-0.029 1.521456220024313
sto1-0.5785785785785785-0.04804804804804804-0.02402402402402402-0.016016016016016016-0.038038038038038034-0.29529529529529525 0.8534658080660665
sto1-0.2152152152152152-0.3973973973973974-0.02102102102102102-0.018018018018018014-0.15915915915915915-0.18918918918918917 0.772330766731316
Best responding to: sto1-0.044-0.046-0.063-0.204-0.159-0.484 , with ent: 0.004182685396551122
{0: (0, 1, 0, 0, 0, 0), 1: [0.044, 0.046, 0.063, 0.204, 0.159, 0.484]}
{1: [4], 0: [1]}
[0.05867665418227216, 0.05742821473158552, 0.05867665418227216, 0.19350811485642946, 0.1523096129837703, 0.4794007490636704]
sto1-0.06693306693306694-0.12687312687312688-0.2667332667332668-0.04695304695304696-0.2577422577422578-0.23476523476523478 0.394093518242786
sto1-0.4645354645354645-0.042957042957042946-0.049950049950049945-0.043956043956043946-0.10889110889110888-0.28971028971028967 0.48408662328235874
sto1-0.1781781781781782-0.3633633633633634-0.09309309309309

sto1-0.04304304304304304-0.050050050050050046-0.27727727727727725-0.06806806806806806-0.3643643643643643-0.1971971971971972 0.43150418243790917
sto1-0.6796796796796797-0.10910910910910909-0.027027027027027025-0.050050050050050046-0.09709709709709709-0.03703703703703703 1.4384142953911236
sto1-0.347-0.029-0.277-0.04-0.03-0.277 0.6540540974530835
sto1-0.281-0.096-0.134-0.295-0.108-0.086 0.6339017858869475
sto1-0.242-0.182-0.053-0.133-0.192-0.198 0.3263261164991574
sto1-0.044-0.046-0.063-0.204-0.159-0.484 0.003476025892654678
sto1-0.073-0.032-0.459-0.089-0.281-0.066 0.9129728729582168
sto1-0.141-0.006-0.137-0.096-0.246-0.374 0.2080390731671975
sto1-0.037-0.016-0.234-0.052-0.06-0.601 0.29639472492737157
sto1-0.078-0.43-0.038-0.006-0.218-0.23 0.8672399259815171
sto1-0.050050050050050046-0.02302302302302302-0.3973973973973974-0.22922922922922923-0.11711711711711711-0.18318318318318316 0.4165795809497338
sto1-0.06-0.304-0.044-0.454-0.136-0.002 2.4291739402989365
sto1-0.221-0.062-0.137-0.3-0.0

sto1-0.073-0.032-0.459-0.089-0.281-0.066 0.9040089422952013
sto1-0.141-0.006-0.137-0.096-0.246-0.374 0.2076918638841535
sto1-0.037-0.016-0.234-0.052-0.06-0.601 0.29797244398585676
sto1-0.078-0.43-0.038-0.006-0.218-0.23 0.8587191900765196
sto1-0.050050050050050046-0.02302302302302302-0.3973973973973974-0.22922922922922923-0.11711711711711711-0.18318318318318316 0.4142789196543225
sto1-0.06-0.304-0.044-0.454-0.136-0.002 2.415209762719322
sto1-0.221-0.062-0.137-0.3-0.026-0.254 0.3622762154798508
sto1-0.038-0.683-0.074-0.045-0.131-0.029 1.5217914464369957
sto1-0.5785785785785785-0.04804804804804804-0.02402402402402402-0.016016016016016016-0.038038038038038034-0.29529529529529525 0.8593687599172728
sto1-0.2152152152152152-0.3973973973973974-0.02102102102102102-0.018018018018018014-0.15915915915915915-0.18918918918918917 0.7753497021593112
Best responding to: sto1-0.044-0.046-0.063-0.204-0.159-0.484 , with ent: 0.0032122902250831804
{0: (0, 1, 0, 0, 0, 0), 1: [0.044, 0.046, 0.063, 0.204, 0.1

sto1-0.221-0.062-0.137-0.3-0.026-0.254 0.365400033517709
sto1-0.038-0.683-0.074-0.045-0.131-0.029 1.5326750237465503
sto1-0.5785785785785785-0.04804804804804804-0.02402402402402402-0.016016016016016016-0.038038038038038034-0.29529529529529525 0.8595730563032419
sto1-0.2152152152152152-0.3973973973973974-0.02102102102102102-0.018018018018018014-0.15915915915915915-0.18918918918918917 0.7787975262324657
Best responding to: sto1-0.044-0.046-0.063-0.204-0.159-0.484 , with ent: 0.0030080350877190443
{0: (0, 1, 0, 0, 0, 0), 1: [0.044, 0.046, 0.063, 0.204, 0.159, 0.484]}
{1: [4], 0: [1]}
[0.05433526011560694, 0.056647398843930635, 0.06127167630057803, 0.18959537572254334, 0.15260115606936417, 0.48554913294797686]
sto1-0.06693306693306694-0.12687312687312688-0.2667332667332668-0.04695304695304696-0.2577422577422578-0.23476523476523478 0.39035572661852014
sto1-0.4645354645354645-0.042957042957042946-0.049950049950049945-0.043956043956043946-0.10889110889110888-0.28971028971028967 0.490961386717

[0.05304740406320542, 0.05756207674943566, 0.062076749435665914, 0.18848758465011287, 0.1523702031602709, 0.48645598194130923]
sto1-0.06693306693306694-0.12687312687312688-0.2667332667332668-0.04695304695304696-0.2577422577422578-0.23476523476523478 0.3879686385924112
sto1-0.4645354645354645-0.042957042957042946-0.049950049950049945-0.043956043956043946-0.10889110889110888-0.28971028971028967 0.4929458177656556
sto1-0.1781781781781782-0.3633633633633634-0.0930930930930931-0.04504504504504505-0.08808808808808809-0.23223223223223227 0.5174924125418419
sto1-0.018-0.358-0.066-0.437-0.089-0.032 1.1955986274674706
sto1-0.04304304304304304-0.050050050050050046-0.27727727727727725-0.06806806806806806-0.3643643643643643-0.1971971971971972 0.42460680185658906
sto1-0.6796796796796797-0.10910910910910909-0.027027027027027025-0.050050050050050046-0.09709709709709709-0.03703703703703703 1.450844387142005
sto1-0.347-0.029-0.277-0.04-0.03-0.277 0.6607286703401748
sto1-0.281-0.096-0.134-0.295-0.108-0.0

In [19]:
e.game

<Game values:[[0.902, 0.902], [0.9790000000000001, 0.9790000000000001], [0.848, 0.848], [0.976, 0.976], [0.956, 0.956], [0.912, 0.912]] players{1: <StochasticAttacker id:1 resources:1>, 0: <DirichletMultinomialDefender id:0 resources:1>} time_horizon:100>

In [28]:
e.agent.belief.loglk

{<StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.6848977664046223,
 <StochasticAttacker id:1 resources:1>: -1.6094379124340925,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.688879454114018,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.688879454114018,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -3.190303301163861,
 <StochasticAttacker id:1 resources:1>: -1.2740214327993638,
 <StochasticAttacker id:1 resources:1>: -0.09126790561131946,
 <StochasticAttacker id:1 resources:1>: -3.6848977664046223,
 <StochasticAttacker id:1 resources:1>: -0.1133238910766163,
 <StochasticAttacker id:1 resources:1>: -3.32124264636016,
 <StochasticAttacker id:1 resources:1>: -1.6956695963863746,
 <Stochasti

In [29]:
gm1 = deepcopy(batches[0].configurations[1].game)

In [30]:
gm1

<GameWithObservabilities values:[[5.0, 5.0], [3.0, 3.0], [8.0, 8.0], [4.0, 4.0], [9.0, 9.0]] players{1: <StackelbergAttacker id:1 resources:1>, 0: <FB id:0 resources:1>} time_horizon:1000>

In [31]:
e1 = runner.Experiment(gm1)

In [32]:
for i in range(900):
    e1.run_interaction_with_observations()
    print(e1.game.strategy_history[-1])
    #print(util.game_str(e.game, lenght=7))
    print(e1.game.history[-1])
    print(e1.game.observation_history[-1])
    #print(list(e.agent.belief.pr.items())[2])

{0: (0, 0, 0, 1, 0), 1: [0, 0, 0, 0, 1]}
{1: [4], 0: [3]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: (0, 0, 0, 0, 1), 1: [0, 0, 1, 0, 0]}
{1: [2], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}


{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 0, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 0, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 0, 1: 1, 2: 0, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 

{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 1, 2: 1, 3: 0, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 1, 2: 1, 3: 1, 4: 0}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [2]}
{0: 1, 1: 0, 2: 0, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [0]}
{0: 1, 1: 0, 2: 1, 3: 1, 4: 1}
{0: [0.0828025477706999, 0.0, 0.42675159235668786, 0.0, 0.4904458598726123], 1: [1, 0, 0, 0, 0]}
{1: [0], 0: [4]}
{0: 1, 1: 0, 2: 

In [33]:
e1.game

<GameWithObservabilities values:[[5.0, 5.0], [3.0, 3.0], [8.0, 8.0], [4.0, 4.0], [9.0, 9.0]] players{1: <StackelbergAttacker id:1 resources:1>, 0: <FB id:0 resources:1>} time_horizon:1000>

In [34]:
e1.agent.belief.loglk

{<StochasticAttacker id:1 resources:1>: -1.1129739779822934,
 <StochasticAttacker id:1 resources:1>: -3.688879454114005,
 <StochasticAttacker id:1 resources:1>: -3.31473542646758,
 <StochasticAttacker id:1 resources:1>: -3.676243002079115,
 <StochasticAttacker id:1 resources:1>: -0.10840989264820851,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -1.6094379124340936,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: None,
 <StackelbergAttacker id:1 resources:1>: 0.0,
 <StochasticAttacker id:1 resources:1>: -3.676243002079115,
 <StochasticAttacker id:1 resources:1>: -0.8422254363670488,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -1.0222401724074046,
 <StochasticAttacker id:1 resources:1>: None,
 <StochasticAttacker id:1 resources:1>: -1.5353064195098325,
 <StochasticAttacker id:1 resources:1>: -0.13063341972759857,
 <Stochast